In [1]:
import numpy as np
import random
from numpy.random import randn

In [2]:
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0], [1], [1], [0]])

In [3]:
class GenAl:
    def __init__(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.hidden_weights = np.random.randn(self.num_inputs, self.num_hidden)
        self.output_weights = np.random.randn(self.num_hidden, self.num_outputs)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        hidden_layer = self.sigmoid(np.dot(inputs, self.hidden_weights))
        output_layer = self.sigmoid(np.dot(hidden_layer, self.output_weights))
        return output_layer

    def calculate_fitness(self, X, y):
        predictions = self.predict(X)
        fitness = np.sum(np.abs(predictions - y))
        return fitness

    def crossover(self, other):
        child = GenAl(self.num_inputs, self.num_hidden, self.num_outputs)
        child.hidden_weights = np.where(np.random.rand(*self.hidden_weights.shape) > 0.5, self.hidden_weights, other.hidden_weights)
        child.output_weights = np.where(np.random.rand(*self.output_weights.shape) > 0.5, self.output_weights, other.output_weights)
        return child

    def mutate(self, mutation_rate):
        if np.random.rand() < mutation_rate:
            self.hidden_weights += np.random.normal(0, 0.1, self.hidden_weights.shape)
        if np.random.rand() < mutation_rate:
            self.output_weights += np.random.normal(0, 0.1, self.output_weights.shape)

    def train(self, X, y, population_size=10, generations=100, mutation_rate=0.1):
        population = [self] * population_size

        for generation in range(generations):
            fitnesses = [individual.calculate_fitness(X, y) for individual in population]
            fittest_idx = np.argmin(fitnesses)
            fittest = population[fittest_idx]

            if fitnesses[fittest_idx] == 0:
                print("Solution found in generation", generation)
                break

            parents = np.random.choice(population, size=population_size-1, replace=True, p=np.array(fitnesses) / np.sum(fitnesses))
            children = [fittest.crossover(parent) for parent in parents]
            population = [fittest] + children
            for individual in population:
                individual.mutate(mutation_rate)

        self.hidden_weights = fittest.hidden_weights
        self.output_weights = fittest.output_weights


In [4]:
net1 = GenAl(3,2,1)
net1.train(X, y)

In [5]:
predictions = np.round(net1.predict(X))
accuracy = np.mean(predictions == y)
print("Accuracy:", accuracy)

Accuracy: 0.75


In [6]:
class CulAl:
    def __init__(self, num_inputs, num_hidden, num_outputs, num_knowledges, knowledge_threshold):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.num_knowledges = num_knowledges
        self.knowledge_threshold = knowledge_threshold
        self.hidden_weights = np.random.randn(self.num_inputs, self.num_hidden)
        self.output_weights = np.random.randn(self.num_hidden, self.num_outputs)
        self.knowledges = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        hidden_layer = self.sigmoid(np.dot(inputs, self.hidden_weights))
        output_layer = self.sigmoid(np.dot(hidden_layer, self.output_weights))
        return output_layer

    def calculate_fitness(self, X, y):
        predictions = self.predict(X)
        fitness = np.sum(np.abs(predictions - y))
        return fitness

    def update_knowledge(self, X, y):
        fitness = self.calculate_fitness(X, y)
        if len(self.knowledges) < self.num_knowledges or fitness < max([k["fitness"] for k in self.knowledges]):
            self.knowledges.append({"hidden_weights": self.hidden_weights.copy(), "output_weights": self.output_weights.copy(), "fitness": fitness})
        self.knowledges = sorted(self.knowledges, key=lambda k: k["fitness"])[:self.num_knowledges]

    def inherit_knowledge(self):
        if len(self.knowledges) > 0 and self.calculate_fitness(X, y) > self.knowledge_threshold:
            best_knowledge = self.knowledges[0]
            self.hidden_weights = best_knowledge["hidden_weights"].copy()
            self.output_weights = best_knowledge["output_weights"].copy()

    def train(self, X, y, population_size=10, generations=100, mutation_rate=0.1):
        population = [CulAl(self.num_inputs, self.num_hidden, self.num_outputs, self.num_knowledges, self.knowledge_threshold) for _ in range(population_size)]

        for generation in range(generations):
            for individual in population:
                individual.update_knowledge(X, y)

            for individual in population:
                individual.inherit_knowledge()

            fitnesses = [individual.calculate_fitness(X, y) for individual in population]
            fittest_idx = np.argmin(fitnesses)
            fittest = population[fittest_idx]

            if fitnesses[fittest_idx] == 0:
                print("Solution found in generation", generation)
                break

            parents = np.random.choice(population, size=population_size-1, replace=True, p=np.array(fitnesses) / np.sum(fitnesses))
            children = [fittest.crossover(parent) for parent in parents]
            population = [fittest] + children
            for individual in population:
                individual.mutate(mutation_rate)

        self.hidden_weights = fittest.hidden_weights
        self.output_weights = fittest.output_weights

    def crossover(self, other):
        child = CulAl(self.num_inputs, self.num_hidden, self.num_outputs, self.num_knowledges, self.knowledge_threshold)
        child.hidden_weights = self.hidden_weights.copy()
        child.output_weights = other.output_weights.copy()
        return child

    def mutate(self, mutation_rate):
        for i in range(self.num_inputs):
            for j in range(self.num_hidden):
                if np.random.random() < mutation_rate:
                    self.hidden_weights[i,j] += np.random.randn() * 0.1

        for i in range(self.num_hidden):
            for j in range(self.num_outputs):
                if np.random.random() < mutation_rate:
                    self.output_weights[i,j] += np.random.randn() * 0.1


In [7]:
net2 = CulAl(3,2,1,5,0.5)
net2.train(X, y)

In [8]:
predictions = np.round(net2.predict(X))
accuracy = np.mean(predictions == y)
print("Accuracy:", accuracy)

Accuracy: 0.75


In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [10]:
class PSO:
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.W1 = np.random.randn(self.input_dim, self.hidden_dim)
        self.W2 = np.random.randn(self.hidden_dim, self.output_dim)
        

    def forward(self, X):
        self.z1 = np.dot(X, self.W1)
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2)
        self.y_hat = sigmoid(self.z2)
        return self.y_hat
    
    def backward(self, X, y, y_hat):
        delta3 = (y_hat - y) * sigmoid_derivative(y_hat)
        dW2 = np.dot(self.a1.T, delta3)
        delta2 = np.dot(delta3, self.W2.T) * sigmoid_derivative(self.a1)
        dW1 = np.dot(X.T, delta2)
        return dW1, dW2
    
    def train(self, X, y, iterations, swarm_size, c1, c2, w):
        self.iterations = iterations
        self.swarm_size = swarm_size
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.best_weights = np.zeros(self.W1.size + self.W2.size)
        self.best_error = np.inf
        particles = np.random.uniform(-1, 1, (self.swarm_size, self.W1.size + self.W2.size))
        velocities = np.zeros((self.swarm_size, self.W1.size + self.W2.size))
        for i in range(self.iterations):
            for j in range(self.swarm_size):
                self.W1 = particles[j][:self.W1.size].reshape(self.input_dim, self.hidden_dim)
                self.W2 = particles[j][self.W1.size:].reshape(self.hidden_dim, self.output_dim)
                y_hat = self.forward(X)
                error = np.mean(np.square(y - y_hat))
                if error < self.best_error:
                    self.best_weights = particles[j].copy()
                    self.best_error = error
                dW1, dW2 = self.backward(X, y, y_hat)
                velocity = velocities[j] * self.w + self.c1 * np.random.rand() * (self.best_weights - particles[j]) + self.c2 * np.random.rand() * (self.best_weights - particles[j])
                particles[j] += velocity
        self.W1 = self.best_weights[:self.W1.size].reshape(self.input_dim, self.hidden_dim)
        self.W2 = self.best_weights[self.W1.size:].reshape(self.hidden_dim, self.output_dim)

In [11]:
net3 = PSO(input_dim=3, hidden_dim=4, output_dim=1)

In [12]:
net3.train(X, y, iterations=100, swarm_size=10, c1=2, c2=2, w=0.5)

/var/folders/zx/mhdl9f0d1c552plym17cmqym0000gn/T/ipykernel_26800/3668515781.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [13]:
predictions = np.round(net3.forward(X))
accuracy = np.mean(predictions == y)
print("Accuracy:", accuracy)

Accuracy: 1.0


/var/folders/zx/mhdl9f0d1c552plym17cmqym0000gn/T/ipykernel_26800/3668515781.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [14]:
class ACO:
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.W1 = np.random.randn(self.input_dim, self.hidden_dim)
        self.W2 = np.random.randn(self.hidden_dim, self.output_dim)
        
    def forward(self, X):
        self.z1 = np.dot(X, self.W1)
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2)
        self.y_hat = sigmoid(self.z2)
        return self.y_hat
    
    def backward(self, X, y, y_hat):
        delta3 = (y_hat - y) * sigmoid_derivative(y_hat)
        dW2 = np.dot(self.a1.T, delta3)
        delta2 = np.dot(delta3, self.W2.T) * sigmoid_derivative(self.a1)
        dW1 = np.dot(X.T, delta2)
        return dW1, dW2
    
    def train(self, X, y, iterations, num_ants, evaporation_rate, alpha, beta, Q):
        self.iterations = iterations
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.Q = Q
        self.best_weights = np.zeros(self.W1.size + self.W2.size)
        self.best_error = np.inf
        pheromones = np.ones((self.W1.size + self.W2.size, 2))
        for i in range(self.iterations):
            for j in range(self.num_ants):
                ant_weights = np.zeros(self.W1.size + self.W2.size)
                for k in range(len(ant_weights)):
                    if np.random.rand() < pheromones[k][1]:
                        ant_weights[k] = np.random.uniform(-1, 1)
                    else:
                        ant_weights[k] = self.best_weights[k]
                self.W1 = ant_weights[:self.W1.size].reshape(self.input_dim, self.hidden_dim)
                self.W2 = ant_weights[self.W1.size:].reshape(self.hidden_dim, self.output_dim)
                y_hat = self.forward(X)
                error = np.mean(np.square(y - y_hat))
                if error < self.best_error:
                    self.best_weights = ant_weights.copy()
                    self.best_error = error
                dW1, dW2 = self.backward(X, y, y_hat)
                pheromones_update = np.zeros((self.W1.size + self.W2.size, 2))
                for k in range(len(pheromones_update)):
                    if ant_weights[k] == self.best_weights[k]:
                        pheromones_update[k][0] = (1 - self.evaporation_rate) * pheromones[k][0] + self.Q / self.best_error
                        pheromones_update[k][1] = self.alpha / (self.alpha + self.beta)
                    else:
                        pheromones_update[k][0] = (1 - self.evaporation_rate) * pheromones[k][0]
                        pheromones_update[k][1] = self.beta / (self.alpha + self.beta)
                pheromones = pheromones_update
                self.W1 -= self.alpha * dW1
                self.W2 -= self.alpha * dW2
    
    def predict(self, X):
        return self.forward(X)

In [15]:
net4 = ACO(input_dim=3, hidden_dim=4, output_dim=1)

In [16]:
net4.train(X, y, iterations=100, num_ants=10, evaporation_rate=0.2, alpha=0.1, beta=0.9, Q=10)

In [17]:
predictions = np.round(net4.forward(X))
accuracy = np.mean(predictions == y)
print("Accuracy:", accuracy)

Accuracy: 0.5
